# Criando um corretor ortográfico

### Importando as bibliotecas:

In [1]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rafaelmaacario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Dados de treinamento

### 1.1 Leitura do corpus de artigos da Alura:  

In [2]:
file_path = "/Users/rafaelmaacario/Corretor Ortográfico em Python/corretor-master/artigos.txt"

with open(file_path, "r") as file:
    corpus = file.read()

corpus[:500]

'\n\n\nimagem \n\nTemos a seguinte classe que representa um usuário no nosso sistema:\n\njava\n\nPara salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:\n\njava \n\nSuponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou'

### 1.2 Limpeza dos dados: 

In [3]:
def tokenize(text):
    list_tokens = nltk.word_tokenize(text)
    return list_tokens

def lista_palavras(list_tokens):
    lista_palavras = [token.lower() for token in list_tokens if token.isalpha()]
    return lista_palavras

In [4]:
lista_palavras = lista_palavras(tokenize(corpus))
print(lista_palavras[:50])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', 'java', 'para', 'salvar', 'um', 'novo', 'usuário', 'várias', 'validações', 'são', 'feitas', 'como', 'por', 'exemplo', 'ver', 'se', 'o', 'nome', 'só', 'contém', 'letras', 'o', 'cpf', 'só', 'números', 'e', 'ver', 'se', 'o', 'usuário', 'possui', 'no', 'mínimo', 'anos', 'veja', 'o', 'método', 'que', 'faz']


In [5]:
frequencia_palavras_corpus = nltk.FreqDist(lista_palavras)
frequencia_palavras_corpus.most_common(10)

[('de', 15502),
 ('o', 14056),
 ('que', 12230),
 ('a', 11099),
 ('e', 10501),
 ('para', 7710),
 ('um', 6367),
 ('é', 5899),
 ('uma', 5220),
 ('do', 5124)]

In [6]:
total_palavras = len(lista_palavras)
total_palavras

403031

In [7]:
# Vocabulário real do corpus:
    
vocabulário = set(lista_palavras)
len(vocabulário)

18464

## 2. Funções de correção

### 2.1. Corrigindo erros por operações de inserção:

In [113]:
palavra = 'cmpo'

(palavra[:1], palavra[1:])

('c', 'mpo')

In [114]:
def corretor_por_insercao(palavra):
    
    # Dividir a palavra em tuplas:
    tuplas = [(palavra[:i], palavra[i:]) for i in range(len(palavra) + 1)]
        
    # Inserir letras dentro das tuplas:
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    novas_palavras = [tupla_esq + letra + tupla_dir for tupla_esq, tupla_dir in tuplas for letra in letras]
    
    # Calcular a probabilidade de cada palavra gerada com base na lista de palavras do corpus:
    probabilidade_palavras = [frequencia_palavras_corpus[palavra_gerada] / total_palavras for palavra_gerada in novas_palavras]
    
    # Encontrar a palavra correta de maior probabilidade:
    palavra_correta = max(zip(novas_palavras, probabilidade_palavras), key=lambda x: x[1])[0]
    
    return [palavra_correta]

In [115]:
corretor_por_insercao('lgica')

['lógica']

### 2.2. Corrigindo erros por operações de alternância de um termo: 

In [116]:
def corretor_por_alterancia(palavra):
    
    # Dividir a palavra em tuplas removendo o primeiro caractere após a divisão:
    tuplas = [(palavra[:i], palavra[i+1:]) for i in range(len(palavra) + 1)]
    
    # Inserir letras dentro das tuplas:
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    novas_palavras = [tupla_esq + letra + tupla_dir for tupla_esq, tupla_dir in tuplas for letra in letras]
    
    # Calcular a probabilidade de cada palavra gerada com base na lista de palavras do corpus:
    probabilidade_palavras = [frequencia_palavras_corpus[palavra_gerada] / total_palavras for palavra_gerada in novas_palavras]
    
    # Encontrar a palavra correta de maior probabilidade:
    palavra_correta = max(zip(novas_palavras, probabilidade_palavras), key=lambda x: x[1])[0]
    
    return [palavra_correta] 

In [117]:
corretor_por_alterancia('lígica')

['lógica']

###  2.3. Corrigindo erros por operações que deletam até dois caracteres extras seguidos: 

In [132]:
def corretor_deleta_caracter_extra(palavra):
    # Dividir a palavra em tuplas removendo o primeiro caracter após a divisão:
    tuplas = [(palavra[:i], palavra[i+1:]) for i in range(len(palavra) + 1)]
    
    # Juntar a tupla
    novas_palavras = [tupla_esq + tupla_dir for tupla_esq, tupla_dir in tuplas]
    
    # Calcular a probabilidade de cada palavra gerada com base na lista de ocorrência de palavras do corpus:
    probabilidade_palavras = [frequencia_palavras_corpus[palavra_gerada] / total_palavras for palavra_gerada in novas_palavras]
    
    # Encontrar a palavra correta de maior probabilidade:
    palavra_correta = max(zip(novas_palavras, probabilidade_palavras), key=lambda x: x[1])[0]
    
    return [palavra_correta] 

In [135]:
palavra = 'lóigica'

In [136]:
corretor_deleta_caracter_extra(palavra)

['lógica']

In [152]:
def corretor_deleta_dois_caracteres_extras_seguidos(palavra):
    iteracao = 0
    while True:
        # Dividir a palavra em tuplas removendo até dois caracteres após a divisão:
        tuplas = [(palavra[:i], palavra[i+min(iteracao, 2):]) for i in range(len(palavra) + 1)]

        # Juntar as tuplas
        novas_palavras = [tupla_esq + tupla_dir for tupla_esq, tupla_dir in tuplas]

        # Encontrar candidatas a palavra correta:
        candidatas = [palavra_gerada for palavra_gerada in novas_palavras]

        if any(candidata in vocabulário for candidata in candidatas):
            # Calcular a probabilidade apenas para as palavras no vocabulário:
            probabilidade_palavras = [frequencia_palavras_corpus.get(palavra_gerada, 0) / total_palavras for palavra_gerada in candidatas]

            # Encontrar a palavra correta de maior probabilidade:
            palavra_correta = max(zip(candidatas, probabilidade_palavras), key=lambda x: x[1])[0]

            return [palavra_correta]

        iteracao += 1

In [153]:
corretor_deleta_dois_caracteres_extras_seguidos('dosss')

['dos']

In [154]:
corretor_deleta_dois_caracteres_extras_seguidos('dooos')

['dos']

###  2.4. Corrigindo erros por operações que invertem a ordem de dois caracteres: 

In [122]:
def corretor_inverte_caracteres(palavra):
    # Dividir a palavra em tuplas:
    tuplas = [(palavra[:i], palavra[i:]) for i in range(len(palavra) + 1)]
    
    # Inserir letras dentro das tuplas:
    novas_palavras = [tupla_esq + (tupla_dir[1] + tupla_dir[0] + tupla_dir[2:] if tupla_dir and len(tupla_dir) > 1 else '') for tupla_esq, tupla_dir in tuplas]

    # Calcular a probabilidade de cada palavra gerada com base na lista de palavras do corpus:
    probabilidade_palavras = [frequencia_palavras_corpus[palavra_gerada] / total_palavras for palavra_gerada in novas_palavras]
    
    # Encontrar a palavra correta de maior probabilidade:
    palavra_correta = max(zip(novas_palavras, probabilidade_palavras), key=lambda x: x[1])[0]  
    
    return palavra_correta

In [123]:
corretor_inverte_caracteres('lgóica')

'lógica'

## 3. Dados para teste 

In [124]:
test_path = "/Users/rafaelmaacario/Corretor Ortográfico em Python/corretor-master/palavras.txt"

with open(test_path, "r") as f:
    lista_palavras_teste = [tuple(linha.split()) for linha in f]

f.close()
lista_palavras_teste[:20]

[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm'),
 ('ele', 'eme'),
 ('fazer', 'èazer'),
 ('temos', 'temfs'),
 ('essa', 'eàssa'),
 ('quando', 'quaôdo'),
 ('vamos', 'vamvos'),
 ('sobre', 'hsobre'),
 ('java', 'sjava'),
 ('das', 'daõs'),
 ('agora', 'agorah')]

### 3.1 Função de avaliação do corretor: 

In [137]:
def avaliador(lista_palavras_teste, vocabulario):
    numero_palavras = len(lista_palavras_teste)
    acertos = 0
    desconhecidas = 0

    for certa, errada in lista_palavras_teste:

        palavra_corrigida = []
        palavra_corrigida.extend(corretor_por_insercao(errada))
        palavra_corrigida.extend(corretor_por_alterancia(errada))
        palavra_corrigida.extend(corretor_deleta_caracter_extra(errada))
        palavra_corrigida.extend(corretor_inverte_caracteres(errada))
        
        
        if certa in palavra_corrigida:
            acertos += 1
        elif certa not in vocabulario:
            desconhecidas +=1 

    taxa_acertos = round(acertos * 100 / numero_palavras, 2)
    taxa_desconhecida = round(desconhecidas * 100 / numero_palavras, 2)

    return print(f"{taxa_acertos}% de {numero_palavras} palavras.\n {taxa_desconhecida}% de {numero_palavras} palavras.")

In [138]:
avaliador(lista_palavras_teste, vocabulário)

79.57% de 186 palavras.
 6.99% de 186 palavras.
